In [1]:
import pandas as pd
import numpy as np
import requests
import json
import config
import io
import os
import dotenv
import sys
sys.tracebacklimit = 0

import geopandas as gpd
from geopandas import GeoDataFrame, points_from_xy
from cartoframes import read_carto, to_carto
from cartoframes.auth import set_default_credentials, Credentials
from cartoframes.viz import Map, Layer, basemaps
from cartoframes.data.clients import SQLClient

PROJ: proj_create_from_database: Cannot find proj.db


In [2]:
credentials = pd.read_csv('credentials.csv',header=0)
username = credentials.columns[0]
api_key = credentials.values[0][0]

set_default_credentials(username=username,api_key=api_key)

credentials = Credentials(username=username,api_key=api_key)

sql = SQLClient(credentials)

In [3]:
zipper = pd.read_csv('Miami Zip codes.csv')
zips = zipper['Zip Code'].tolist()
zips.sort()

In [4]:
dotenv.load_dotenv()
key = os.getenv('ACSkey')

### Means of Transportation to Work

In [5]:
tran = pd.DataFrame()

## Zip Code level geography not supported prior to 2011
for y in range(2011, 2020):
    
    HOST = "https://api.census.gov/data"
    year = str(y)
    dataset = 'acs/acs5'
    type_ = '?get='
    base_url = "/".join([HOST, year, dataset, type_])#, get_vars, location])
    
    var = 'NAME,B08301_001E,B08301_003E,B08301_004E,B08301_010E,B08301_016E,B08301_017E,B08301_018E,B08301_019E,'\
            'B08301_020E,B08301_021E'
    
    loc = '&for=zip%20code%20tabulation%20area:'+','.join(map(str,zips))+'&for=county:086&in=state:12'
    
    k = '&key='+key
    
    full_url = base_url + var + loc + k
    
    data = requests.get(full_url).content
    df1 = pd.read_csv(io.StringIO(data.decode('utf-8')))
    df1 = df1.replace('\]','',regex=True).replace('\[','',regex=True).replace('\"','',regex=True)
    df1 = df1.iloc[:,:-1]
    df1['year'] = y
    col_names = ['zcta','total','drove','carpool','publictrans','taxi','motorbike','bike','walk','other','wfh',
                 'state','zip','year']
    
    for i in range(0,len(col_names),1):
        df1.rename(columns={df1.columns[i]:col_names[i]},inplace=True) 
    tran = tran.append(df1,ignore_index=True)
## For some reason 2019 data being added twice
## remove using drop_duplicates
## Should be 720 rows when done (80 zips by 9 years)
tran.drop_duplicates(inplace=True)
## drop superfluous columns
tran.drop(['zcta', 'state'], axis=1, inplace=True)

In [5]:
## Export to csv
tran.to_csv("tran.csv", sep=",", index=False)

In [18]:
## Push to Carto
to_carto(tran, 'tran', if_exists='replace')

Success! Data uploaded to table "tran" correctly


'tran'

### Additional Data to Collect

* Housing Unit Value/Median Area Housing Value (Zillow access; got Census data)
* Travel time to work
* X National Area Risk Index (FEMA) - wasn't fully released until July 2021
* X Social Vulnerability Index (CDC) - only available at non-consistent intervals (2010,2014,2016,2018)

### Labor Force Participation Rate

In [6]:
lfp = pd.DataFrame()

## Zip Code level geography not supported prior to 2011
for y in range(2011, 2020):
    
    HOST = "https://api.census.gov/data"
    year = str(y)
    dataset = 'acs/acs5'
    type_ = '?get='
    base_url = "/".join([HOST, year, dataset, type_])#, get_vars, location])
    
    var = 'NAME,B23025_001E,B23025_002E,B23025_003E,B23025_004E,B23025_005E,B23025_006E,B23025_007E'
    
    loc = '&for=zip%20code%20tabulation%20area:'+','.join(map(str,zips))+'&for=county:086&in=state:12'
    
    k = '&key='+key
    
    full_url = base_url + var + loc + k
    
    data = requests.get(full_url).content
    df1 = pd.read_csv(io.StringIO(data.decode('utf-8')))
    df1 = df1.replace('\]','',regex=True).replace('\[','',regex=True).replace('\"','',regex=True)
    df1 = df1.iloc[:,:-1]
    df1['year'] = y
    col_names = ['zcta','pop','labforce','civlf','emp','unemp','military','notinlf','state','zip','year']

    for i in range(0,len(col_names),1):
        df1.rename(columns={df1.columns[i]:col_names[i]},inplace=True) 
    lfp = lfp.append(df1,ignore_index=True)

lfp.drop_duplicates(inplace=True)
## Create labor force participation rate variable
lfp['part_rate'] = lfp['labforce']/lfp['pop']
## Create civilian force participation rate variable
lfp['civ_rate'] = lfp['civlf']/lfp['pop']
## drop superfluous columns
lfp.drop(['zcta', 'state'], axis=1, inplace=True)

In [7]:
## Export to csv
lfp.to_csv("lfp.csv", sep=",", index=False)

In [8]:
## Push to Carto
to_carto(lfp, 'lfp', if_exists='replace')

Success! Data uploaded to table "lfp" correctly


'lfp'

### Educational Attainment

In [7]:
ed = pd.DataFrame()

## Zip Code level geography not supported prior to 2011
for y in range(2011, 2020):
    
    HOST = "https://api.census.gov/data"
    year = str(y)
    dataset = 'acs/acs5'
    type_ = '?get='
    base_url = "/".join([HOST, year, dataset, type_])#, get_vars, location])
    
    ## 2nd var is total pop 16 years & over
    ## these bachelor stats are for ages 25 to 64
    var = 'NAME,B23025_001E,B23006_023E,B23006_024E,B23006_025E,B23006_026E,B23006_027E,B23006_028E,B23006_029E'
    
    loc = '&for=zip%20code%20tabulation%20area:'+','.join(map(str,zips))+'&for=county:086&in=state:12'
    
    k = '&key='+key
    
    full_url = base_url + var + loc + k
    
    data = requests.get(full_url).content
    df1 = pd.read_csv(io.StringIO(data.decode('utf-8')))
    df1 = df1.replace('\]','',regex=True).replace('\[','',regex=True).replace('\"','',regex=True)
    df1 = df1.iloc[:,:-1]
    df1['year'] = y
    col_names = ['zcta','tot','bach','bachlf','bachmil','bachciv','bachemp','bachunemp','bachnotinlf','state','zip','year']

    for i in range(0,len(col_names),1):
        df1.rename(columns={df1.columns[i]:col_names[i]},inplace=True) 
    ed = ed.append(df1,ignore_index=True)

ed.drop_duplicates(inplace=True)
## Create bachelor's in labor force rate variable
ed['per_bachlf'] = ed['bachlf']/ed['tot']
## Create bachelor's in civilian labor force rate variable
ed['per_bachciv'] = ed['bachciv']/ed['tot']
## drop superfluous columns
ed.drop(['zcta', 'state'], axis=1, inplace=True)

In [9]:
## Export to csv
ed.to_csv("ed.csv", sep=",", index=False)

In [10]:
## Push to Carto
to_carto(ed, 'ed', if_exists='replace')

Success! Data uploaded to table "ed" correctly


'ed'

### Field of Bachelor's Degree

In [8]:
maj = pd.DataFrame()

## Zip Code level geography not supported prior to 2013
for y in range(2013, 2020):
    
    HOST = "https://api.census.gov/data"
    year = str(y)
    dataset = 'acs/acs5'
    type_ = '?get='
    base_url = "/".join([HOST, year, dataset, type_])#, get_vars, location])
    
    ## total bachelor's here includes for all ages
    var = 'NAME,B15012_001E,B15012_002E,B15012_003E,B15012_004E,B15012_005E,B15012_006E,B15012_007E,' \
    'B15012_008E,B15012_009E,B15012_010E,B15012_011E,B15012_012E,B15012_013E,B15012_014E,B15012_015E,B15012_016E'
    
    loc = '&for=zip%20code%20tabulation%20area:'+','.join(map(str,zips))+'&for=county:086&in=state:12'
    
    k = '&key='+key
    
    full_url = base_url + var + loc + k
    
    data = requests.get(full_url).content
    df1 = pd.read_csv(io.StringIO(data.decode('utf-8')))
    df1 = df1.replace('\]','',regex=True).replace('\[','',regex=True).replace('\"','',regex=True)
    df1 = df1.iloc[:,:-1]
    df1['year'] = y
    col_names = ['zcta','tot_bach','pc_math_stat','bio_ag_es','phys_rel_sci','psych','social_sci','eng','multi_sci',
                 'relat_fields','biz','educ','lit_lang','lib_hist','viz_perf','comm','other_art_hum','state','zip','year']

    for i in range(0,len(col_names),1):
        df1.rename(columns={df1.columns[i]:col_names[i]},inplace=True) 
    maj = maj.append(df1,ignore_index=True)

maj.drop_duplicates(inplace=True)
## Add science & engineering degrees
maj['sci_eng'] = maj.iloc[:, 2:10].sum(axis=1)
## Add arts & humanities degrees
maj['arts_hum'] = maj.iloc[:, 12:17].sum(axis=1)
## drop superfluous columns
maj.drop(['zcta', 'state'], axis=1, inplace=True)

In [11]:
## Export to csv
maj.to_csv("maj.csv", sep=",", index=False)

In [12]:
## Push to Carto
to_carto(maj, 'maj', if_exists='replace')

Success! Data uploaded to table "maj" correctly


'maj'

### Internet Access

In [9]:
net = pd.DataFrame()

## Zip Code level geography not supported prior to 2017
for y in range(2017, 2020):
    
    HOST = "https://api.census.gov/data"
    year = str(y)
    dataset = 'acs/acs5'
    type_ = '?get='
    base_url = "/".join([HOST, year, dataset, type_])#, get_vars, location])
    
    ## 3rd variable (device) counts any computing device, inlcuding smartphones
    ## 4th variable (pc) only counts if desktop or laptop
    var = 'NAME,B28001_001E,B28001_002E,B28001_003E,B28002_002E,B28002_006E'
    
    loc = '&for=zip%20code%20tabulation%20area:'+','.join(map(str,zips))+'&for=county:086&in=state:12'
    
    k = '&key='+key
    
    full_url = base_url + var + loc + k
    
    data = requests.get(full_url).content
    df1 = pd.read_csv(io.StringIO(data.decode('utf-8')))
    df1 = df1.replace('\]','',regex=True).replace('\[','',regex=True).replace('\"','',regex=True)
    df1 = df1.iloc[:,:-1]
    df1['year'] = y
    col_names = ['zcta','households','device','pc','internet','cell_data','state','zip','year']

    for i in range(0,len(col_names),1):
        df1.rename(columns={df1.columns[i]:col_names[i]},inplace=True) 
    net = net.append(df1,ignore_index=True)

net.drop_duplicates(inplace=True)
## Percent of households columns
net['per_pc'] = net['pc']/net['households']
net['per_int'] = net['internet']/net['households']
## Internet not including cellular data plan
net['int_noCell'] = net['internet'] - net['cell_data']
net['per_int_noCell'] = net['int_noCell']/net['households']
## drop superfluous columns
net.drop(['zcta', 'state', 'cell_data'], axis=1, inplace=True)

In [13]:
## Export to csv
net.to_csv("net.csv", sep=",", index=False)

In [14]:
## Push to Carto
to_carto(net, 'net', if_exists='replace')

Success! Data uploaded to table "net" correctly


'net'

### Housing Statistics

In [5]:
hom = pd.DataFrame()

## Zip Code level geography not supported prior to 2017
for y in range(2011, 2020):
    
    HOST = "https://api.census.gov/data"
    year = str(y)
    dataset = 'acs/acs5'
    type_ = '?get='
    base_url = "/".join([HOST, year, dataset, type_])#, get_vars, location])
    
    #B25123_001E same as B25002_002E (occupied units)
    var = 'NAME,B25077_001E,B25035_001E,B25001_001E,B25002_002E,B25002_003E,B25123_002E,B25123_007E,B25123_008E,B25123_013E'
    
    loc = '&for=zip%20code%20tabulation%20area:'+','.join(map(str,zips))+'&for=county:086&in=state:12'
    
    k = '&key='+key
    
    full_url = base_url + var + loc + k
    
    data = requests.get(full_url).content
    df1 = pd.read_csv(io.StringIO(data.decode('utf-8')))
    df1 = df1.replace('\]','',regex=True).replace('\[','',regex=True).replace('\"','',regex=True)
    df1 = df1.iloc[:,:-1]
    df1['year'] = y
    col_names = ['zcta','medval','medyear','tot_units','occup','vacant','own_occ','own_std',
                 'rent_occ','rent_std','state','zip','year']

    for i in range(0,len(col_names),1):
        df1.rename(columns={df1.columns[i]:col_names[i]},inplace=True) 
    hom = hom.append(df1,ignore_index=True)

hom.drop_duplicates(inplace=True)
## Substandard households columns
hom['substd'] = hom['occup'] - (hom['own_std']+hom['rent_std'])
## drop superfluous columns
hom.drop(['zcta', 'state'], axis=1, inplace=True)

In [21]:
## Export to csv
hom.to_csv("hom.csv", sep=",", index=False)

In [16]:
## Push to Carto
to_carto(hom, 'hom', if_exists='replace')

Success! Data uploaded to table "hom" correctly


'hom'

### 

### Income & PopDense

In [5]:
inc = pd.DataFrame()

## Zip Code level geography not supported prior to 2011
for y in range(2011, 2020):
    
    HOST = "https://api.census.gov/data"
    year = str(y)
    dataset = 'acs/acs5'
    type_ = '?get='
    base_url = "/".join([HOST, year, dataset, type_])#, get_vars, location])
    
    var = 'NAME,B17026_001E,B19001_001E,B19013_001E,B19025_001E,B19083_001E,B19101_001E,B19113_001E,B19301_001E,B01003_001E'
    
    loc = '&for=zip%20code%20tabulation%20area:'+','.join(map(str,zips))+'&for=county:086&in=state:12'
    
    k = '&key='+key
    
    full_url = base_url + var + loc + k
    
    data = requests.get(full_url).content
    df1 = pd.read_csv(io.StringIO(data.decode('utf-8')))
    df1 = df1.replace('\]','',regex=True).replace('\[','',regex=True).replace('\"','',regex=True)
    df1 = df1.iloc[:,:-1]
    df1['year'] = y
    col_names = ['zcta','Inc2Pov','housInc','MedhousInc','agghousinc','gini','famInc','MedfamInc','incPercap','pop','state','zip','year']

    for i in range(0,len(col_names),1):
        df1.rename(columns={df1.columns[i]:col_names[i]},inplace=True) 
    inc = inc.append(df1,ignore_index=True)

inc.drop_duplicates(inplace=True)
## drop superfluous columns
inc.drop(['zcta', 'state'], axis=1, inplace=True)

In [6]:
area = pd.read_csv('zipArea.csv')
area = area[['ZCTA','LandSqMi']]
area = area[area['ZCTA'].isin(zips)].reset_index(drop=True)
area.rename({'ZCTA':'zip'},axis=1,inplace=True)
area['zip'] = area.zip.astype(str)
inc = inc.merge(area, how='outer', on='zip')
inc['meanhousinc'] = inc['agghousinc']/inc['housInc']
inc['popDens'] = inc['pop']/inc['LandSqMi']
inc.drop(['pop', 'LandSqMi'], axis=1, inplace=True)

In [10]:
## Export to csv
inc.to_csv("inc.csv", sep=",", index=False)

In [11]:
## Push to Carto
to_carto(inc, 'inc', if_exists='replace')

Success! Data uploaded to table "inc" correctly


'inc'

In [30]:
## URL output loop to check pulling correctly

In [14]:
lfp = pd.DataFrame()

## Zip Code level geography not supported prior to 2011
for y in range(2011, 2012):
    
    HOST = "https://api.census.gov/data"
    year = str(y)
    dataset = 'acs/acs5'
    type_ = '?get='
    base_url = "/".join([HOST, year, dataset, type_])#, get_vars, location])
    
    var = 'NAME,B23025_001E,B23025_002E,B23025_003E,B23025_004E,B23025_005E,B23025_006E,B23025_007E'
    
    loc = '&for=zip%20code%20tabulation%20area:'+','.join(map(str,zips))+'&for=county:086&in=state:12'
    
    k = '&key='+key
    
    full_url = base_url + var + loc + k
    
    print(full_url)

https://api.census.gov/data/2011/acs/acs5/?get=NAME,B17026_001E,B19001_001E,B19013_001E,B19025_001E,B19083_001E,B19101_001E,B19113_001E,B19301_001E,B01003_001E&for=zip%20code%20tabulation%20area:33010,33012,33013,33014,33015,33016,33018,33030,33031,33032,33033,33034,33035,33039,33054,33055,33056,33101,33109,33122,33125,33126,33127,33128,33129,33130,33131,33132,33133,33134,33135,33136,33137,33138,33139,33140,33141,33142,33143,33144,33145,33146,33147,33149,33150,33154,33155,33156,33157,33158,33160,33161,33162,33165,33166,33167,33168,33169,33170,33172,33173,33174,33175,33176,33177,33178,33179,33180,33181,33182,33183,33184,33185,33186,33187,33189,33190,33193,33194,33196&for=county:086&in=state:12&key=5ac177d9ef065948cac225addd7cec7ad4945850
